In [1]:
import tensorflow as tf
import os
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import boto3
from PIL import Image
import io

s3 = boto3.resource('s3')
bucket = 'capstone-trafficlights'
classes = ['Red', 'Yellow', 'Green']

In [2]:
from tensorflow.contrib.layers import flatten

def LeNet(x):    
    # Hyperparameters
    mu = 0
    sigma = 0.1
    
    # SOLUTION: Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b

    # SOLUTION: Activation.
    conv1 = tf.nn.relu(conv1)

    # SOLUTION: Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Layer 2: Convolutional. Output = 10x10x16.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # SOLUTION: Activation.
    conv2 = tf.nn.relu(conv2)

    # SOLUTION: Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

    # SOLUTION: Flatten. Input = 5x5x16. Output = 400.
    fc0   = flatten(conv2)
    
    # SOLUTION: Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # SOLUTION: Activation.
    fc1    = tf.nn.relu(fc1)

    # SOLUTION: Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
    
    # SOLUTION: Activation.
    fc2    = tf.nn.relu(fc2)

    # SOLUTION: Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, num_classes), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(num_classes))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [3]:
num_classes =3
x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, num_classes)

In [4]:
rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for X_batch, y_batch in val_generator(data):
        accuracy = sess.run(accuracy_operation, feed_dict={x: X_batch, y: y_batch})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [5]:
from sklearn.utils import shuffle

def load_data_from_s3(path):
    data =[]

    with open('./csv_classes.csv') as csvfile:
        reader = csv.reader(csvfile)
        line_count = 0
        for line in reader:
            if line_count>0:
                image = line[0]
                color = line[1]
                data.append(([image,color]))
               
            line_count+=1
    return data
def get_image(bucket,sample,resize):
    try:
        key= sample[0]
        image_obj = s3.Object(bucket, key)
        image = io.BytesIO(image_obj.get()['Body'].read())
        image = np.asarray(Image.open(image).resize((resize,resize)))
#         image= np.asarray( Image.open(image), dtype="int32" )
        return image
    except Exception as err:
        print(err)

def generator(bucket,samples,n_classes, batch_size = 32, resize=32):
    while True:
        for offset in range(0,len(samples), batch_size):
            start = offset
            end = offset+batch_size
            batch_sample = samples[start:end]
            images=[]
            colors=[]
            for sample in batch_sample:
                #get images
                image = get_image(bucket, sample, resize)
                images.append(image)
                colors.extend(sample[1])

            x = np.array(images)
            y = np.array(colors)
            yield shuffle(x,y)        




In [12]:
resize = 32
num_classes =3


data = load_data_from_s3(bucket)
train_data, validation_data = train_test_split(data, test_size=0.2)

train_generator = generator(bucket, train_data,num_classes , resize=resize)
val_generator = generator(bucket, validation_data,num_classes, resize=resize)
dumdum, dumdum2 = train_generator.next()
print(dumdum[0].shape)

(32, 32, 3)


In [ ]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print("Training...")
    for i in range(2):  
        for X_batch, y_batch in train_generator:
            sess.run(training_operation, feed_dict={x: X_batch, y: y_batch})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, 'lenet')
    print("Model saved")

Training...
